# Deep Learning for Text - Encoder Models

## Data Preprocessing



Let's begin extracting the data from the ATIS dataset and turning into a form that we can use in our Deep Learning models.

The ATIS dataset is standard benchmark dataset widely used to build models for intent classification and slot filling tasks (we will explain all this shortly). You can find a very detailed explanation [here](https://catalog.ldc.upenn.edu/docs/LDC93S4B/corpus.html).

We will begin by loading the file and then partitioning into a test and a training set.

In [15]:
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import warnings

import tensorflow as tf
from tensorflow import keras


tf.random.set_seed(42)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [16]:
# Read in the training and testing data
df_train = pd.read_csv('sample_data/atis_train_data.csv')
df_test = pd.read_csv('sample_data/atis_test_data.csv')

Let's visualize all of this on a dataframe. Below we display an example query for each intent class in a nice layout.

The first column of the Dataframe below contains the actual query that was asked. The second column indicates the intent (flight, flight time, etc), whereas the last column contains the slot filling structure.

In [17]:
# create the sample dataframe with examples of intent to df_small so you can visualize the training inputs
##df_small = df_test.filter(['query','intent','slot filling'])
pd.set_option('display.max_colwidth', None)  # fix -1 -> None
df_small = pd.DataFrame(columns=['query','intent','slot filling'])
j = 0
for i in df_train.intent.unique():
  df_small.loc[j] = df_train[df_train.intent==i].iloc[0]
  j += 1
df_small

,query,intent,slot filling
0,i want to fly from boston at 838 am and arrive in denver at 1110 in the morning,flight,O O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O B-arrive_time.period_of_day
1,what is the arrival time in san francisco for the 755 am flight leaving washington,flight_time,O O O B-flight_time I-flight_time O B-fromloc.city_name I-fromloc.city_name O O B-depart_time.time I-depart_time.time O O B-fromloc.city_name
2,cheapest airfare from tacoma to orlando,airfare,B-cost_relative O O B-fromloc.city_name O B-toloc.city_name
3,what kind of aircraft is used on a flight from cleveland to dallas,aircraft,O O O O O O O O O O B-fromloc.city_name O B-toloc.city_name
4,what kind of ground transportation is available in denver,ground_service,O O O O O O O O B-city_name
5,what 's the airport at orlando,airport,O O O O O B-city_name
6,which airline serves denver pittsburgh and atlanta,airline,O O O B-fromloc.city_name B-fromloc.city_name O B-fromloc.city_name
7,how far is it from orlando airport to orlando,distance,O O O O O B-fromloc.airport_name I-fromloc.airport_name O B-toloc.city_name
8,what is fare code h,abbreviation,O O O O B-fare_basis_code
9,how much does the limousine service cost within pittsburgh,ground_fare,O O O O B-transport_type O O O B-city_name


Let's see how many different types of "intent" are present in the data.

In [18]:
# Get the counts for all intents of the train datasets and
# assign it to variable "intent_counts" to be printed
intent_counts = df_train['intent'].value_counts()
intent_counts

intent
flight                        3666
airfare                        423
ground_service                 255
airline                        157
abbreviation                   147
aircraft                        81
flight_time                     54
quantity                        51
flight+airfare                  21
airport                         20
distance                        20
city                            19
ground_fare                     18
capacity                        16
flight_no                       12
meal                             6
restriction                      6
airline+flight_no                2
ground_service+ground_fare       1
airfare+flight_time              1
cheapest                         1
aircraft+flight+flight_no        1
Name: count, dtype: int64

In [19]:
# Extract query_data_train, intent_data_train, slot_data_train,
#    query_data_test, intent_data_test, slot_data_test from the train and test dataframes
query_data_train = df_train['query'].values
intent_data_train = df_train['intent'].values
slot_data_train = df_train['slot filling'].values

query_data_test = df_test['query'].values
intent_data_test = df_test['intent'].values
slot_data_test = df_test['slot filling'].values

We briefly mentioned what the difference were between slot filling and intent in the introduction, but is worth going into more detail.

As an example, let’s consider the user query “*i want to fly from boston at 838 am and arrive in denver at 1110 in the morning*”. The model should classify this user query as “**flight**” intent. It should also parse the query, identify and fill all slots necessary for understanding the query. Although the words “I”, “want”, “to”, “fly”, “from”, “at”, “and”, “arrive”, “in”, “the” contribute to understand the context of the intent, the model should correctly label the entities needed to fulfill user’s goal in its intention to take a flight. These are “boston” as departure city (B-fromloc.city), “8:38 am” as departure time (B-depart_time.time), “denver” as destination city (B-toloc.city_name), “11:10” as arrival time (B-arrive_time.time) and “morning” as arrival period of day (B-arrive_time.period_of_day). The 123 slot categories are shown below.

In [20]:
# Gather all the unique slots and put them in the set called "unique_slots"
unique_slots = set()
for s in slot_data_train:
  unique_slots = unique_slots.union(set(s.split()))
unique_slots

{'B-aircraft_code',
 'B-airline_code',
 'B-airline_name',
 'B-airport_code',
 'B-airport_name',
 'B-arrive_date.date_relative',
 'B-arrive_date.day_name',
 'B-arrive_date.day_number',
 'B-arrive_date.month_name',
 'B-arrive_date.today_relative',
 'B-arrive_time.end_time',
 'B-arrive_time.period_mod',
 'B-arrive_time.period_of_day',
 'B-arrive_time.start_time',
 'B-arrive_time.time',
 'B-arrive_time.time_relative',
 'B-city_name',
 'B-class_type',
 'B-connect',
 'B-cost_relative',
 'B-day_name',
 'B-day_number',
 'B-days_code',
 'B-depart_date.date_relative',
 'B-depart_date.day_name',
 'B-depart_date.day_number',
 'B-depart_date.month_name',
 'B-depart_date.today_relative',
 'B-depart_date.year',
 'B-depart_time.end_time',
 'B-depart_time.period_mod',
 'B-depart_time.period_of_day',
 'B-depart_time.start_time',
 'B-depart_time.time',
 'B-depart_time.time_relative',
 'B-economy',
 'B-fare_amount',
 'B-fare_basis_code',
 'B-flight_days',
 'B-flight_mod',
 'B-flight_number',
 'B-flight_st

In [21]:
len(unique_slots)

123

**123 slot categories!!**

## Transformers





### Explain the attention mechanism



### Encoder Model



Because the code for transformer encoder architecture is a bit complicated to write, we have decided to package it. This means that you can import it directly from our own "library" (in the same way you do it for Keras layers).

Import `TransformerEncoder`, and `PositionalEmbedding` from 'HODL' file present in the current file directory.

Hint: Take a look at the `HODL.py` file on the left-sidebar menu.

In [22]:
# Import the transformer encoder and the positional embedding
from sample_data.HODL import TransformerEncoder, PositionalEmbedding

In [23]:
query_data_train[:5]

array([' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning ',
       ' what flights are available from pittsburgh to baltimore on thursday morning ',
       ' what is the arrival time in san francisco for the 755 am flight leaving washington ',
       ' cheapest airfare from tacoma to orlando ',
       ' round trip fares from pittsburgh to philadelphia under 1000 dollars '],
      dtype=object)

In [24]:
slot_data_train[:5]

array([' O O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O B-arrive_time.period_of_day ',
       ' O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day ',
       ' O O O B-flight_time I-flight_time O B-fromloc.city_name I-fromloc.city_name O O B-depart_time.time I-depart_time.time O O B-fromloc.city_name ',
       ' B-cost_relative O O B-fromloc.city_name O B-toloc.city_name ',
       ' B-round_trip I-round_trip O O B-fromloc.city_name O B-toloc.city_name B-cost_relative B-fare_amount I-fare_amount '],
      dtype=object)

In [25]:
# Set the max_query_length to 30 to start
max_query_length=30

# Textvec of slots. Assign this to "text_vectorization_slots"
text_vectorization_slots = keras.layers.TextVectorization(
    output_sequence_length=max_query_length,
    standardize=None
)

# Adapt the slot training data
text_vectorization_slots.adapt(slot_data_train)

# Assign the number of slots to slot_vocab_size
slot_vocab_size = text_vectorization_slots.vocabulary_size()

# Get the "target_train" and "target_test" data
target_train = text_vectorization_slots(slot_data_train)
target_test = text_vectorization_slots(slot_data_test)

# Get the text_vectorization_query
text_vectorization_query = keras.layers.TextVectorization(
    output_sequence_length=max_query_length
)

# Adapt the query train data
text_vectorization_query.adapt(query_data_train)

# Assign the number of unique query words
# Words vectorization
query_vocab_size = text_vectorization_query.vocabulary_size()

# Create source_train and source_test
source_train = text_vectorization_query(query_data_train)
source_test = text_vectorization_query(query_data_test)


In [26]:
# Create the 4 model params
embedding_dim = 512
encoder_units = 64
units = 128
num_heads = 8

# Embedding and Masking
# Create the inputs, embedding, and x
inputs = keras.Input(shape=(max_query_length,))
embedding = PositionalEmbedding(max_query_length, query_vocab_size, embedding_dim)
x = embedding(inputs)

# Transformer Encoding
# Create encoder_out
encoder_out = TransformerEncoder(embedding_dim, encoder_units, num_heads)(x)

# Classifier
# add layers to x and define the outputs
x = keras.layers.Dense(units, activation='relu')(encoder_out)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(slot_vocab_size, activation="softmax")(x)

### finally apply the inputs and outputs to your model
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 positional_embedding (Posi  (None, 30, 512)           470016    
 tionalEmbedding)                                                
                                                                 
 transformer_encoder (Trans  (None, 30, 512)           8469568   
 formerEncoder)                                                  
                                                                 
 dense_2 (Dense)             (None, 30, 128)           65664     
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 dense_3 (Dense)             (None, 30, 125)           16125 

In [27]:
# Compile your model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])

In [28]:
# Set the batch size to 64 and epochs to 10 to start
batch_size = 64
epochs = 10

# Fit the model
model.fit(x=source_train,
          y=target_train,
          epochs=epochs
          )

Epoch 1/10
156/156 [==============================] - 15s 50ms/step - loss: 0.3861 - sparse_categorical_accuracy: 0.9198
Epoch 2/10
156/156 [==============================] - 4s 26ms/step - loss: 0.1160 - sparse_categorical_accuracy: 0.9645
Epoch 3/10
156/156 [==============================] - 4s 26ms/step - loss: 0.0742 - sparse_categorical_accuracy: 0.9774
Epoch 4/10
156/156 [==============================] - 4s 26ms/step - loss: 0.0501 - sparse_categorical_accuracy: 0.9858
Epoch 5/10
156/156 [==============================] - 4s 23ms/step - loss: 0.0366 - sparse_categorical_accuracy: 0.9896
Epoch 6/10
156/156 [==============================] - 4s 26ms/step - loss: 0.0288 - sparse_categorical_accuracy: 0.9915
Epoch 7/10
156/156 [==============================] - 4s 28ms/step - loss: 0.0242 - sparse_categorical_accuracy: 0.9927
Epoch 8/10
156/156 [==============================] - 4s 23ms/step - loss: 0.0223 - sparse_categorical_accuracy: 0.9935
Epoch 9/10
156/156 [===================

In [29]:
# define slot_filling_accuracy function
from sklearn.metrics import accuracy_score
def slot_filling_accuracy(actual, predicted, only_slots=True):
    not_padding = np.not_equal(actual, 0)
    if only_slots:
        non_slot_token = text_vectorization_slots(['O']).numpy()[0, 0]
        slots = np.not_equal(actual, non_slot_token)
        correct_predictions = np.equal(actual, predicted)[not_padding * slots]
    else:
        correct_predictions = np.equal(actual, predicted)[not_padding]

    sample_length = len(correct_predictions)
    weights = np.ones(sample_length)
    return np.dot(correct_predictions, weights) / sample_length


# Get the predicted data
predicted = np.argmax(model.predict(source_test), axis=-1).reshape(-1)

# Get the actual data
actual = target_test.numpy().reshape(-1)

# Now get the accuracy "acc" and slot accuracy "acc_slots"
acc = slot_filling_accuracy(actual, predicted, only_slots=False)
acc_slots = slot_filling_accuracy(actual, predicted, only_slots=True)

print(f'Accuracy = {acc:.3f}')
print(f'Accuracy on slots = {acc_slots:.3f}')

28/28 [==============================] - 1s 11ms/step
Accuracy = 0.969
Accuracy on slots = 0.928


Now we get 92% accuracy on the slots and 97% accuracy in general. This is so much better!!

Let's see some examples:

In [32]:
# Define the predict_slots_query() function which takes in a query
def predict_slots_query(query):
    sentence = text_vectorization_query([query])
    prediction = np.argmax(model.predict(sentence), axis=-1)[0]
    inverse_vocab = dict(enumerate(text_vectorization_slots.get_vocabulary()))
    return " ".join(inverse_vocab[int(i)] for i in prediction)

examples = [
            'from los angeles',
            'to los angeles',
            'from boston',
            'to boston',
            'cheapest flight from boston to los angeles tomorrow',
            'what is the airport at orlando',
            'what are the air restrictions on flights from pittsburgh to atlanta for the airfare of 416 dollars',
            'flight from boston to santiago',
            'flight boston to santiago'
]

for e in examples:
    print(e,"\n",predict_slots_query(e),"\n")

1/1 [==============================] - 0s 19ms/step
from los angeles 
 O B-fromloc.city_name I-fromloc.city_name                            

1/1 [==============================] - 0s 19ms/step
to los angeles 
 O B-toloc.city_name I-toloc.city_name                            

1/1 [==============================] - 0s 21ms/step
from boston 
 O B-fromloc.city_name                             

1/1 [==============================] - 0s 21ms/step
to boston 
 O B-toloc.city_name                             

1/1 [==============================] - 0s 32ms/step
cheapest flight from boston to los angeles tomorrow 
 B-cost_relative O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name B-depart_date.today_relative                       

1/1 [==============================] - 0s 32ms/step
what is the airport at orlando 
 O O O O O B-toloc.city_name                         

1/1 [==============================] - 0s 36ms/step
what are the air restrictions on flights from pittsburgh to at

Even though 'Santiago' is not a city that is present in the training data set, it is still capable of recognizing it as a destination city name just by context! This is the power of the attention mechanism of transformers.

Can we get even better accuracy if we train for longer? Let's try!

In [33]:
# Try to change parameters to get your accuracy up
batch_size = 128
epochs = 30

# Fit your model
model.fit(x=source_train,
          y=target_train,
          epochs=epochs
          )

Epoch 1/30
156/156 [==============================] - 4s 24ms/step - loss: 0.0174 - sparse_categorical_accuracy: 0.9946
Epoch 2/30
156/156 [==============================] - 4s 24ms/step - loss: 0.0157 - sparse_categorical_accuracy: 0.9952
Epoch 3/30
156/156 [==============================] - 4s 26ms/step - loss: 0.0181 - sparse_categorical_accuracy: 0.9946
Epoch 4/30
156/156 [==============================] - 4s 24ms/step - loss: 0.0155 - sparse_categorical_accuracy: 0.9956
Epoch 5/30
156/156 [==============================] - 4s 24ms/step - loss: 0.0154 - sparse_categorical_accuracy: 0.9952
Epoch 6/30
156/156 [==============================] - 4s 26ms/step - loss: 0.0155 - sparse_categorical_accuracy: 0.9956
Epoch 7/30
156/156 [==============================] - 4s 24ms/step - loss: 0.0144 - sparse_categorical_accuracy: 0.9958
Epoch 8/30
156/156 [==============================] - 4s 25ms/step - loss: 0.0138 - sparse_categorical_accuracy: 0.9959
Epoch 9/30
156/156 [====================

In [34]:
# Define slot_filling_accuracy
def slot_filling_accuracy_(actual, predicted, only_slots=True):
    not_padding = np.not_equal(actual, 0)
    if only_slots:
        non_slot_token = text_vectorization_slots(['O']).numpy()[0, 0]
        slots = np.not_equal(actual, non_slot_token)
        correct_predictions = np.equal(actual, predicted)[slots*not_padding]
    else:
        correct_predictions = np.equal(actual, predicted)[not_padding]
    sample_length = len(correct_predictions)
    return np.sum(correct_predictions) / sample_length

predicted = np.argmax(model.predict(source_test), axis=-1).reshape(-1)
actual = target_test.numpy().reshape(-1)

acc = slot_filling_accuracy(actual, predicted, only_slots=False)
acc_slots = slot_filling_accuracy(actual, predicted, only_slots=True)

print(f'Accuracy = {acc:.3f}')
print(f'Accuracy on slots = {acc_slots:.3f}')


28/28 [==============================] - 0s 11ms/step
Accuracy = 0.972
Accuracy on slots = 0.937
